Equal-Weight S&P 500 Index Fund
Introduction & Library Imports
The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial


In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

Importing Our List of Stocks
The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course.

There's a static version of the S&P 500 constituents available here. Click this link to download them now. Move this file into the starter-files folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks =pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Acquiring an API Token
Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a secrets.py file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

Click here to download your secrets.py file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secret import IEX_CLOUD_API_TOKEN

Making Our First API Call
Now it's time to structure our API calls to IEX cloud.

We need the following information from the API:

Market capitalization for each stock
Price of each stock

In [4]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/'
print(api_url)

https://sandbox.iexapis.com/stable/stock/AAPL/quote/


In [5]:
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
print(api_url)

https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448


In [6]:
data=requests.get(api_url)
type(data)
print(data.status_code!=200)

False


In [7]:
data=data.json()
print(data)

{'avgTotalVolume': 102302783, 'calculationPrice': 'close', 'change': -4.32, 'changePercent': -0.03021, 'close': 142.9, 'closeSource': 'cffliiao', 'closeTime': 1680835792169, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 145, 'delayedPriceTime': 1694223048704, 'extendedChange': -0.67, 'extendedChangePercent': -0.0048, 'extendedPrice': 142.44, 'extendedPriceTime': 1708004625450, 'high': 149.3, 'highSource': ' t1eradneey c iedumlp5i', 'highTime': 1716341536938, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 140.19, 'iexCloseTime': 1737633153036, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 141.83, 'iexOpenTime': 1683456231822, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1716520213948, 'latestPrice': 140.6, 'latestSource': 'Close', 'latestTime': 'September 30, 2022', 'latestUpdate': 1675671667694, 'latestVolume': 129349722, 'low': 140, 'lowSource': '1m5ep aeiydnicted u

In [8]:
dictionary={
    'a':1,
    'b':2
}
dictionary['a']

1

Parsing Our API Call
The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy.

With that said, the data isn't in a proper format yet. We need to parse it first.

In [9]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
print(data)

{'avgTotalVolume': 105051155, 'calculationPrice': 'close', 'change': -4.38, 'changePercent': -0.03025, 'close': 143, 'closeSource': 'lfciioaf', 'closeTime': 1720621661859, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 143, 'delayedPriceTime': 1721858750777, 'extendedChange': -0.66, 'extendedChangePercent': -0.0047, 'extendedPrice': 143.45, 'extendedPriceTime': 1739905029593, 'high': 145.5, 'highSource': 'yn1ca5ed ered lium ptei', 'highTime': 1731089575705, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 138.14, 'iexCloseTime': 1707590305907, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 142.91, 'iexOpenTime': 1670750778902, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1724600158368, 'latestPrice': 140.4, 'latestSource': 'Close', 'latestTime': 'September 30, 2022', 'latestUpdate': 1667482896081, 'latestVolume': 128979813, 'low': 142, 'lowSource': 't5ine li e1a mucdeydp

In [10]:
price=data['latestPrice']
print(price)

140.4


In [11]:
market_cap=data['marketCap']
print(market_cap)

2259562316224


Adding Our Stocks Data to a Pandas DataFrame
The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [12]:
my_columns=['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [13]:
final_dataframe.append(
    pd.Series(
        [
            symbol,price,market_cap,'N/A'
        ]
    ),
    ignore_index=True
)

C:\Users\Jash\AppData\Local\Temp\ipykernel_10648\1874663306.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,0,1,2,3
0,NaN,NaN,NaN,NaN,AAPL,140.4,2.259562e+12,N/A


In [14]:
final_dataframe.append(
    pd.Series(
        [
            symbol,price,market_cap,'N/A'
        ],
        index=my_columns
    ),
    ignore_index=True
)

C:\Users\Jash\AppData\Local\Temp\ipykernel_10648\549099363.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,140.4,2259562316224,N/A


Looping Through The Tickers in Our List of Stocks
Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a for loop.

In [15]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    print(stock)

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBY
BDX
BEN
BF.B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COO
COP
COST
COTY
CPB
CPRT
CRM
CSCO
CSX
CTAS
CTL
CTSH
CTVA
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
ES
ESS
ETFC
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FLIR
FLS
FLT
FMC
FOX
FOXA
FRC
FRT
FTI
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HBI
HCA
HD
HES
HFC
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRB
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

In [16]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
    api_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    final_dataframe=final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],index=my_columns
        ),
        ignore_index=True
    )

C:\Users\Jash\AppData\Local\Temp\ipykernel_10648\12678084.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe=final_dataframe.append(
C:\Users\Jash\AppData\Local\Temp\ipykernel_10648\12678084.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe=final_dataframe.append(
C:\Users\Jash\AppData\Local\Temp\ipykernel_10648\12678084.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe=final_dataframe.append(
C:\Users\Jash\AppData\Local\Temp\ipykernel_10648\12678084.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe=final_dataframe.append(
C:\Users\Jash\AppData\Local\Temp\ipykernel_1

In [17]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,122.44,37190353850,N/A
1,AAL,12.46,8215347532,N/A
2,AAP,159.16,9678664271,N/A
3,AAPL,143.60,2256186364635,N/A
4,ABBV,140.42,247202156562,N/A


Using Batch API Calls to Improve Performance
Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [18]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[1:i+n]

In [19]:
symbol_groups=list(chunks(stocks['Ticker'],100))
symbol_groups

[1       AAL
 2       AAP
 3      AAPL
 4      ABBV
 5       ABC
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 99, dtype: object,
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
 5       ABC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 199, dtype: object,
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
 5       ABC
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 299, dtype: object,
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
 5       ABC
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 399, dtype: object,
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
 5       ABC
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 499, dtype: object,
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
 5       ABC
      

In [20]:
for i in range(0,len(symbol_groups)):
    print(symbol_groups[i])

1       AAL
2       AAP
3      AAPL
4      ABBV
5       ABC
      ...  
95     CINF
96       CL
97      CLX
98      CMA
99    CMCSA
Name: Ticker, Length: 99, dtype: object
1       AAL
2       AAP
3      AAPL
4      ABBV
5       ABC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 199, dtype: object
1       AAL
2       AAP
3      AAPL
4      ABBV
5       ABC
       ... 
295     MAA
296     MAR
297     MAS
298     MCD
299    MCHP
Name: Ticker, Length: 299, dtype: object
1       AAL
2       AAP
3      AAPL
4      ABBV
5       ABC
       ... 
395     RHI
396     RJF
397      RL
398     RMD
399     ROK
Name: Ticker, Length: 399, dtype: object
1       AAL
2       AAP
3      AAPL
4      ABBV
5       ABC
       ... 
495    XLNX
496     XOM
497    XRAY
498     XRX
499     XYL
Name: Ticker, Length: 499, dtype: object
1       AAL
2       AAP
3      AAPL
4      ABBV
5       ABC
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name

In [21]:
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

In [23]:
# final_dataframe=pd.DataFrame(columns=my_columns)
# for symbol_string in stmbol_strings:
#     batch_api_call_url=f''